In [ ]:
# 🎥 Hands-On: Your First AI Camera Experience

**Time to see AI vision in action!** 🚀

Now that you understand the concepts, let's experience the magic of real-time AI processing with your IMX500 camera.

## 🎯 What You'll Experience

- **Live camera preview** streaming directly to your notebook
- **Real-time performance** of your AI system
- **Interactive controls** to start and stop the preview
- **Preparation** for AI detection in the next level

## 🔧 Camera Preview Setup

Let's start with a basic camera preview to verify everything is working:

In [ ]:
import subprocess
import cv2
import numpy as np
from IPython.display import display, clear_output, Image
import ipywidgets as widgets
import threading
import time

print("🎥 AI CAMERA INITIALIZATION")
print("=" * 30)
print("\n🔧 Preparing your IMX500 camera for preview...")

# ── 0) Kill any stray libcamera processes & unload bcm2835_v4l2 so libcamera has exclusive access
print("🧹 Cleaning up any existing camera processes...")
for p in ("libcamera-vid","libcamera-still","libcamera-raw"):
    subprocess.run(["pkill","-9",p], stderr=subprocess.DEVNULL)
subprocess.run(["sudo","rmmod","bcm2835_v4l2"], stderr=subprocess.DEVNULL)

print("✅ Camera resources prepared")
print("\n🎮 Interactive camera preview controls ready!")
print("💡 Click the button below to start/stop the preview")

In [ ]:
# Stop button in notebook
stopButton = widgets.ToggleButton(
    value=False,
    description='▶️ Start Preview',
    button_style='success',
    icon='play',
    layout=widgets.Layout(width='200px', height='40px')
)

# Status display
status_label = widgets.HTML(
    value="<b>📷 Camera Status:</b> Ready to start",
    layout=widgets.Layout(margin='10px 0px')
)

display(widgets.VBox([status_label, stopButton]))

def camera_preview():
    \"\"\"Main camera preview function\"\"\"
    # Update button appearance when running
    stopButton.description = '⏹️ Stop Preview'
    stopButton.button_style = 'danger'
    stopButton.icon = 'stop'
    status_label.value = "<b>📷 Camera Status:</b> <span style='color: green'>● LIVE</span>"
    
    # Use libcamera-vid to stream MJPEG into stdout
    cmd = [    
        "libcamera-vid",
        "--inline",           # Needed for streaming
        "-t", "0",            # No timeout
        "--width", "640",     # Moderate resolution for good performance
        "--height", "480",
        "--codec", "mjpeg",
        "-o", "-"             # Output to stdout
    ]
    
    try:
        proc = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.DEVNULL)
        
        data = b""
        display_handle = display(None, display_id=True)
        frame_count = 0
        start_time = time.time()
        
        while not stopButton.value:
            chunk = proc.stdout.read(1024)
            if not chunk:
                break
            data += chunk
            a = data.find(b'\\xff\\xd8')  # JPEG start
            b = data.find(b'\\xff\\xd9')  # JPEG end
            if a != -1 and b != -1:
                jpg = data[a:b+2]
                data = data[b+2:]
                frame = cv2.imdecode(np.frombuffer(jpg, dtype=np.uint8), cv2.IMREAD_COLOR)
                
                # Add frame counter and FPS overlay
                frame_count += 1
                if frame_count % 30 == 0:  # Update every 30 frames
                    elapsed = time.time() - start_time
                    fps = frame_count / elapsed if elapsed > 0 else 0
                    status_label.value = f"<b>📷 Camera Status:</b> <span style='color: green'>● LIVE</span> | Frames: {frame_count} | FPS: {fps:.1f}"
                
                _, jpeg = cv2.imencode('.jpg', frame)
                display_handle.update(Image(data=jpeg.tobytes()))
                
    except Exception as e:
        status_label.value = f"<b>📷 Camera Status:</b> <span style='color: red'>● ERROR - {str(e)}</span>"
    finally:
        proc.terminate()
        # Reset button appearance
        stopButton.description = '▶️ Start Preview'
        stopButton.button_style = 'success'
        stopButton.icon = 'play'
        status_label.value = "<b>📷 Camera Status:</b> Stopped"

def on_button_change(change):
    \"\"\"Handle button state changes\"\"\"
    if change['new']:  # Button was pressed (start preview)
        # Start preview in a separate thread
        threading.Thread(target=camera_preview, daemon=True).start()
    # When button is released, the preview loop will detect stopButton.value=False and stop

stopButton.observe(on_button_change, names='value')

## 🎉 Congratulations - You're Seeing Through AI Eyes!\n\n**What you're experiencing right now is remarkable:**\n\n### 📊 Technical Achievement\n- **Real-time video processing** at 20+ frames per second\n- **Direct sensor-to-notebook streaming** with minimal latency\n- **Professional-grade image quality** from a credit card-sized sensor\n- **Headless operation** perfect for remote development\n\n### 🚀 What's Next: AI Intelligence Layer\n\nRight now you're seeing the **raw visual input** that your AI system will analyze. In Level 2, this same video stream will be enhanced with:\n\n- **🎯 Object Detection**: Boxes around detected people, cars, animals\n- **📊 Confidence Scores**: How certain the AI is about each detection\n- **📍 Precise Coordinates**: Exact pixel locations of every object\n- **🏷️ Classification Labels**: \"Person\", \"Car\", \"Dog\", \"Stop Sign\", etc.\n\n### 💡 Professional Insight\n\n**You've just accomplished what would have required expensive equipment and complex setup just a few years ago.** The IMX500 represents a revolution in accessible AI technology.\n\n**This simple preview is the foundation** for every advanced AI application you'll build in this course.\n\n---\n\n## 🎯 Ready for Real AI?\n\n**Your camera preview is working perfectly!** \n\nTime to add the intelligence layer and see your first real-time object detection.\n\n**Next Level**: `02_First_AI_Detection/` - Transform this video stream into intelligent vision!"